In [ ]:
#Sensitive Encryption key
cryptokey='AIxyLjsFocf-NEuEXRS_VUJ1XEEkf4nE9QQ2cWnhlro='

Installing dependencies

In [ ]:
!pip3 install pillow pydub soundfile numpy opencv-python uuid amodem ffmpeg-python

In [ ]:
!pip3 uninstall ffmpeg

Importing necessary libraries

In [ ]:
import os
from PIL import Image
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import cv2
import ffmpeg
import uuid
from ipywidgets import FileUpload
from cryptography.fernet import Fernet

Defining functions for compression

In [ ]:
def compressPNG(origfile, newfile):
    img=Image.open(origfile)
    img=img.convert("P", palette=Image.ADAPTIVE, colors=256)
    img.save(newfile, optimize=True)

In [ ]:
def compressJPEG(origfile,newfile):
    img=Image.open(origfile)
    img=img.convert("P", palette=Image.ADAPTIVE, colors=256)
    img.save(newfile,"JPEG", optimize=True)

In [ ]:
def compressVid(origfile,newfile):
    os.system(f'ffmpeg -i {origfile} -vf "scale=trunc(iw/4)*2:trunc(ih/4)*2" -c:v libx265 -crf 28 {newfile}')

In [ ]:
def compressWAV(origfile, newfile):
    data, samplerate=sf.read(origfile)
    n=len(data)
    Fs=samplerate
    ch1=np.array([data[i][0] for i in range(n)])
    ch2=np.array([data[i][1] for i in range(n)])
    ch1_Fourier=np.fft.fft(ch1)
    abs_ch1_Fourier=np.absolute(ch1_Fourier[:n//2])
    eps=1e-5
    frequenciesToRemove = (1-eps) * np.sum(abs_ch1_Fourier) < np.cumsum(abs_ch1_Fourier)
    f0=(len(frequenciesToRemove)- np.sum(frequenciesToRemove))*(Fs/2)/(n/2)
    D=int(Fs/f0)
    new_data=data[::D,:]
    sf.write(newfile, new_data, int(Fs/D), 'PCM_16')

In [ ]:
def compressMP3(origfile, newfile):
    tempname1=str(uuid.uuid4())+".wav"
    tempname2=str(uuid.uuid4())+".wav"
    audio=AudioSegment.from_mp3(origfile)
    audio.export(tempname1, format="wav")
    compressWAV(tempname1, tempname2)
    audio2=AudioSegment.from_wav(tempname2)
    audio2.export(newfile, format="mp3")
    os.remove(tempname1)
    os.remove(tempname2)

Set bitrate

In [ ]:
def setBitrate():
    os.environ['BITRATE']='16'

In [ ]:
setBitrate()

Calibration
Check audio output device

In [ ]:
!amodem send --calibrate

Interrupt the kernel when done

In [ ]:
#Variable for filename
global fln
fln=''

Handle upload

In [ ]:
def on_upload_change(change):
    global fln
    if not change.new:
        return
    up = change.owner
    for filename,data in up.value.items():
        with open(filename, 'wb') as f:
            fln=filename
            f.write(data['content'])
    up.value.clear()
    up._counter = 0

upload_btn = FileUpload()
upload_btn.observe(on_upload_change, names='_counter')
upload_btn


In [ ]:
print("Filename: "+fln)

Compress

In [ ]:
ext=os.path.splitext(fln)[1]
newname=str(uuid.uuid4())+ext
if ext=='.png' or ext == '.PNG':
    compressPNG(fln, newname)
if ext=='.jpg' or ext=='.JPG' or ext=='.JPEG' or ext=='.jpeg':
    compressJPEG(fln,newname)
if ext=='.mp4' or ext=='.MP4':
    compressVid(fln, newname)
if ext=='.wav' or ext=='.WAV':
    compressWAV(fln, newname)
if ext=='.mp3' or ext=='.MP3':
    compressMP3(fln,newname)

Use compressed file

In [ ]:
file = open(newname, 'rb')
file_content = file.read()
file.close()

Generating Metadata

In [ ]:
flnlen=len(fln).to_bytes(1,byteorder='big')
metadata=flnlen+fln.encode()
data=metadata+file_content

In [ ]:
data

Encrypt

In [ ]:
fkey = Fernet(cryptokey.encode())
encdata=fkey.encrypt(data)
tempdata=encdata

In [ ]:
tempdata=data

Combine metadata and data and write to a temporary file

In [ ]:
tempname=str(uuid.uuid4())
with open(tempname, 'wb') as f:
    f.write(tempdata)
os.environ['tempname']=tempname

In [ ]:
tempname

In [ ]:
tempsize = os.path.getsize(tempname)
print(f"Attempting to send {tempsize} bytes")
print("Make sure receiver is listening")

Sending

In [ ]:
!echo $BITRATE

In [ ]:
!amodem send -vv -i $tempname

Match checksum with receiver

In [ ]:
!sha512sum $tempname

Cleanup: Delete temporary file and uploaded file

In [ ]:
os.remove(fln)
os.remove(newname)
os.remove(tempname)